%md
<img src="/Workspace/Users/danilorock53@gmail.com/nestle.jpg" alt="Texto alternativo" width="1500">

# Projeto Nestle

**Descrição:** Este notebook extrai os dados da camada raw e envia para camada bronze em formato `Parquet`.

#### Bibliotecas Importadas
Importei somente a função col para fazer o processo.

In [0]:
from pyspark.sql.functions import col

#### Autenticação no Azure Data Lake Gen2
É necessário configurar o acesso ao Azure Data Lake fornecendo a chave da conta de armazenamento:

Substitua:
- `STORAGE_ACCOUNT` pelo nome da sua conta de armazenamento.

- `STORAGE_ACCOUNT_KEY` pela chave de acesso.

In [0]:
spark.conf.set(
    "fs.azure.account.key.storagenestlecase.dfs.core.windows.net", 
    'chave')

In [0]:
# Listar arquivos do Data Lake
dbutils.fs.ls("abfss://raw@storagenestlecase.dfs.core.windows.net/")

[FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BaseCEP.csv', name='BaseCEP.csv', size=24695, modificationTime=1737383396000),
 FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BaseCargos.csv', name='BaseCargos.csv', size=1362, modificationTime=1737383396000),
 FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BaseClientes.csv', name='BaseClientes.csv', size=19009, modificationTime=1737383396000),
 FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BaseFuncionarios.csv', name='BaseFuncionarios.csv', size=99764, modificationTime=1737383396000),
 FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BaseNível.csv', name='BaseNível.csv', size=340, modificationTime=1737383396000),
 FileInfo(path='abfss://raw@storagenestlecase.dfs.core.windows.net/BasePQ.csv', name='BasePQ.csv', size=103983, modificationTime=1737659006000)]

#### Leitura e Carga
Extração e transformação dos arquivos CSV, com envio para a camada Bronze, onde foram convertidos para o formato Parquet.

In [0]:
# Caminhos dos arquivos no Data Lake Gen 2
file_paths = [
    "abfss://camada@storage.dfs.core.windows.net/BaseCargos.csv",
    "abfss://camada@storage.dfs.core.windows.net/BaseCEP.csv",
    "abfss://camada@storage.dfs.core.windows.net/BaseClientes.csv",
    "abfss://camada@storage.dfs.core.windows.net/BaseFuncionarios.csv",
    "abfss://camada@storage.dfs.core.windows.net/BaseNível.csv",
    "abfss://camada@storage.dfs.core.windows.net/BasePQ.csv"
]

# Leitura dos arquivos CSV
base_cargos = spark.read.option("delimiter", ";").csv(file_paths[0], header=True, inferSchema=True)
base_cep = spark.read.option("delimiter", "|").csv(file_paths[1], header=True, inferSchema=True)
base_clientes = spark.read.option("delimiter", ";").csv(file_paths[2], header=True, inferSchema=True)
base_funcionarios = spark.read.option("delimiter", "||").csv(file_paths[3], header=True, inferSchema=True)
base_nivel = spark.read.option("delimiter", "%").csv(file_paths[4], header=True, inferSchema=True)
base_pq = spark.read.option("delimiter", ";").csv(file_paths[5], header=True, inferSchema=True)


# Caminho da camada Bronze no Data Lake Gen2
bronze_path = "abfss://bronze@storagenestlecase.dfs.core.windows.net/bronze/"

# Gravar os DataFrames como Parquet na camada Bronze
base_cargos.write.mode("overwrite").parquet(bronze_path + "BaseCargos.parquet")
base_cep.write.mode("overwrite").parquet(bronze_path + "BaseCEP.parquet")
base_clientes.write.mode("overwrite").parquet(bronze_path + "BaseClientes.parquet")
base_funcionarios.write.mode("overwrite").parquet(bronze_path + "BaseFuncionarios.parquet")
base_nivel.write.mode("overwrite").parquet(bronze_path + "BaseNivel.parquet")
base_pq.write.mode("overwrite").parquet(bronze_path + "BasePQ.parquet")